In [18]:
import pandas as pd
import geopandas as gpd
import os
import json

Download the [OS Open Names dataset](https://osdatahub.os.uk/data/downloads/open/OpenNames) and extract the zip into this directory, it should produce a directory called `opname_csv_gb`.

In [5]:
with open("opname_csv_gb/Doc/OS_Open_Names_Header.csv") as f:
    header = f.readline().split(',')

results = pd.DataFrame()
with os.scandir("opname_csv_gb/Data") as it:
    for entry in it:
        if entry.is_file() and entry.name.endswith(".csv"):
            df = pd.read_csv(entry.path, header=None, names=header)
            subset = df[df['TYPE'] == "populatedPlace"]
            results = pd.concat([results, subset[['NAME1', 'NAME2', 'GEOMETRY_X', 'GEOMETRY_Y']]])

/tmp/ipykernel_1100/3634243814.py:8: DtypeWarning: Columns (4,16,17,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(entry.path, header=None, names=header)
/tmp/ipykernel_1100/3634243814.py:8: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(entry.path, header=None, names=header)
/tmp/ipykernel_1100/3634243814.py:8: DtypeWarning: Columns (4,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(entry.path, header=None, names=header)
/tmp/ipykernel_1100/3634243814.py:8: DtypeWarning: Columns (4,31,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(entry.path, header=None, names=header)
/tmp/ipykernel_1100/3634243814.py:8: DtypeWarning: Columns (4,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(entry.path, header=None, nam

In [6]:
gdf = gpd.GeoDataFrame(results, geometry=gpd.points_from_xy(results['GEOMETRY_X'], results['GEOMETRY_Y'], crs=27700))

In [7]:
gdf = gdf.to_crs(4326)

In [8]:
gdf

,NAME1,NAME2,GEOMETRY_X,GEOMETRY_Y,geometry
0,Church Fenton,NaN,451388,436925,POINT (-1.22079 53.82606)
1,Kellingley,NaN,453010,424128,POINT (-1.19835 53.71089)
2,Half Acres,NaN,442612,425013,POINT (-1.35575 53.71979)
3,Newton,NaN,444690,427852,POINT (-1.32385 53.74514)
4,Chapel Haddlesey,NaN,458289,426091,POINT (-1.11801 53.72797)
...,...,...,...,...,...
12,West Yell,NaN,445549,1183102,POINT (-1.17194 60.52893)
13,Hamnavoe,NaN,449675,1180559,POINT (-1.09741 60.50562)
14,Colvister,NaN,451579,1197025,POINT (-1.05846 60.6532)
15,Aywick,NaN,453140,1186685,POINT (-1.03269 60.56017)


In [13]:
gdf['lat'] = round(gdf.geometry.y, 3)
gdf['lon'] = round(gdf.geometry.x, 3)

In [21]:
output = pd.concat([
    gdf[['NAME1', 'lat', 'lon']], 
    gdf[['NAME2', 'lat', 'lon']].dropna().rename(columns={'NAME2': 'NAME1'})
    ]).to_json(orient='values')
with open("app/places.json", "w") as fd:
    fd.write(output)